In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras 
import tf_resnet as resnet
import  tensorflow.keras.backend as K
import numpy as np
import pickle
import pandas as pd
import random
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,average_precision_score,confusion_matrix,recall_score,accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
import math
import gc
import sys

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
num_classes=15
lstm_count=128

In [ ]:
output = open("D:/罗飞/kitchen_processed_data/"+'xall.p', 'rb')
xall=pd.read_pickle(output,compression=None)
output = open("D:/罗飞/kitchen_processed_data/"+'yall.p', 'rb')
yall=pd.read_pickle(output,compression=None)
output.close()
class_names=['Drinking',"Eating","Standing","Sitting","Walking","Open door and get in","Open door and get out","Washing",
           "Open oven","Close oven","Open cabinet","Close cabinet","Open freezer","Close freezer","No activity"]

In [ ]:
le = preprocessing.LabelEncoder()
yall=le.fit_transform(yall)
num_classes=len(np.unique(yall))
print(np.unique(yall,return_counts=True),num_classes)
xall, yall = shuffle(xall,yall)
print(yall[0:15],xall.shape)
x_train, x_test, y_train, y_test =train_test_split(xall, yall, test_size=0.17)

In [ ]:
def create_model(_input,data_format='channels_last'):
        resnet._handle_dim_ordering()
        ### Convolutional blocks
        conv1 = resnet._conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(model_input)
        pool1 = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        repetitions=[2,2]
        for i, r in enumerate(repetitions):
            block = resnet._residual_block(resnet.basic_block, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = resnet._bn_relu(block)
        block_shape = K.int_shape(block)
        pool2 = layers.AveragePooling2D(pool_size=(block_shape[resnet.ROW_AXIS], block_shape[resnet.COL_AXIS]),
                                strides=(1, 1),name='avpool')(block)
        flatten1 = layers.Flatten()(pool2)
        
        layer_shape=K.int_shape(_input)
        pool_lstm1 = layers.AveragePooling2D(pool_size=(layer_shape[1], 4),
                                strides=(1, 4))(_input)
        print(pool_lstm1.shape)
        pool_lstm1 = layers.Lambda(lambda x: K.squeeze(x, axis= 1))(pool_lstm1)
        lstm = layers.Bidirectional(layers.GRU(lstm_count,return_sequences=True))(pool_lstm1)  
        lstm = layers.Bidirectional(layers.GRU(lstm_count))(lstm)

        flattenbn=layers.BatchNormalization(axis=-1)(flatten1)
        lstm=layers.BatchNormalization(axis=-1)(lstm)
        concat = layers.concatenate([flattenbn, lstm], axis=-1, name ='concat')
    
        ## Softmax Output
        output = layers.Dense(num_classes, activation = 'softmax', name='preds')(concat)
    
        model_output = output
        model =tf.keras.Model(_input, model_output)
        return model
        

In [ ]:
img_rows, img_cols = 80, 80
# The CIFAR10 images are RGB.
img_channels = 2
input_shape = (img_rows, img_cols, img_channels)
model_input = tf.keras.Input(input_shape, name='input')
model=create_model(model_input)
model.summary()

In [ ]:
 model.compile(optimizer=tf.keras.optimizers.Adam(0.0005), loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
batch_size = 128
nb_epoch = 360
class_weight0 = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [ ]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_data=(x_test, y_test),
              shuffle=True)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)

In [ ]:
from sklearn.metrics import f1_score,average_precision_score,confusion_matrix,recall_score,accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib import mlab
import matplotlib
import matplotlib.cm as cm
import itertools
font = {'family' : 'normal',
        'size'   : 22}

matplotlib.rc('font', **font)
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)
    plt.figure(figsize=(24,16))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=60,fontsize=17)
    plt.yticks(tick_marks, classes,fontsize=17)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
y_pred = model.predict(x_test,batch_size=200)
y_pred=np.argmax(y_pred, axis = 1)

print(accuracy_score(y_test,y_pred))
recall=recall_score(y_test, y_pred,average='macro')
print(recall)
print(f1_score(y_test,y_pred,average="macro"))

print(y_pred)
cnf_matrix=confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
np.set_printoptions(precision=1)
class_names=class_name=['Drinking',"Eating","Standing","Sitting","Walking","Open door and get in","Open door and get out","Washing",
           "Open oven","Close oven","Open cabinet","Close cabinet","Open fridge","Close fridge","No activity"]
# Plot non-normalized confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=class_names,
#                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()